In [9]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import folium
from PIL import Image
from io import BytesIO
import geopandas as gpd
from collections import Counter
from calendar import month_abbr


In [10]:
datasets = []

for i in range(2013,2020):
    df = pd.read_csv(f'./datasets/crashes_2012-23/crashes_{i}.csv')
    df = df.dropna(subset=['LONGITUDE', 'LATITUDE'])
    df = df[df['LATITUDE'] != 0]
    df = df[df['LATITUDE'] >= 40]
    df = df[df['LONGITUDE'] != 0]
    df = df[(df['LONGITUDE'] >= -80) & (df['LONGITUDE'] <= -65)]

    datasets.append(df)


/tmp/ipykernel_7302/4255088014.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./datasets/crashes_2012-23/crashes_{i}.csv')
/tmp/ipykernel_7302/4255088014.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./datasets/crashes_2012-23/crashes_{i}.csv')
/tmp/ipykernel_7302/4255088014.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./datasets/crashes_2012-23/crashes_{i}.csv')


In [11]:
# Perform KMeans clustering
def showClusterPlots(data, month, year):
    kmeans = KMeans(n_clusters=9)  # Change the number of clusters to 9
    kmeans.fit(data)
    labels = kmeans.labels_

    # Count occurrences of each label
    label_counts = Counter(labels)

    # Create a Folium map with adjusted center and zoom level for a more zoomed-out view
    map_clusters = folium.Map(location=[np.median(data[:, 0]), np.median(data[:, 1])], zoom_start=11)

    colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'cyan', 'magenta', 'brown']  # Change/add colors for 9 clusters

    # Load GeoJSON data for NYC boroughs
    boroughs_geojson = gpd.read_file('https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=GeoJSON')

    map_clusters = folium.Map(location=[np.median(data[:, 0]), np.median(data[:, 1])], zoom_start=11)

    borough_colors = {
        'Brooklyn': 'blue',
        'Bronx': 'red',
        'Manhattan': 'green',
        'Queens': 'purple',
        'Staten Island': 'orange'
    }

    # Add borough outlines with different colors to the map
    for feature in boroughs_geojson.iterfeatures():
        borough_name = feature['properties']['boro_name']  # Assuming the GeoJSON has a 'boro_name' property
        folium.GeoJson(
            feature,
            name=borough_name,
            style_function=lambda x, color=borough_colors.get(borough_name, 'gray'): {
                'color': color,
                'fillOpacity': 0.16,
                'weight': 2
            }
        ).add_to(map_clusters)


    # Add CircleMarkers for each data point and cluster centers as markers
    for i in range(len(kmeans.cluster_centers_)):
        folium.CircleMarker(
            location=kmeans.cluster_centers_[i],
            radius=label_counts[i]/25,
            color=colors[i],
            fill=True,
            fill_color=colors[i],
            fill_opacity=0.38
        ).add_to(map_clusters)

    # Add markers for cluster centers with the number of data points
    for i, center in enumerate(kmeans.cluster_centers_):
        folium.Marker(
            location=center,
            popup=f'Cluster Center {i+1} - {label_counts[i]} data points',
            icon=folium.Icon(color='black')
        ).add_to(map_clusters)

    # Save the image
    img_data = map_clusters._to_png()
    img = Image.open(BytesIO(img_data))

    img.save(f'./images/clusters/cluster_map_{month}_{year}.png')

In [12]:
for i in range(2013,2020):
    data = datasets[i-2013]
    for j in range(1,13):
        data['CRASH DATE'] = pd.to_datetime(data['CRASH DATE'])
        data_month = data[data["CRASH DATE"].dt.month == j]
        data_month = data_month[['LATITUDE', 'LONGITUDE']].values
        m = str(month_abbr[j])
        showClusterPlots(data_month,m,i)

/home/sankalp/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/sankalp/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/sankalp/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/sankalp/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change fr